# Builder

## O que é?

O padrao Builder oferece uma interface para a criacao de objetos complexos, separando a lógica de criacao do objeto e o objeto em si

## Por quê?

Quando um objeto se torna muito complexo, instanciá-lo por meio de um constructor ou mesmo do Factory Method pode ser pouco conveniente. Para resolver esse problema, este design pattern separa toda a lógica da criacao do objeto em outra classe (Builder), que executa todos os passos necessários antes de retornar o objeto (Product). Cabe ao client (Director) a tarefa de chamar os métodos no Builder. Assim, o processo de criacao do objeto (Product) fica totalmente separado do objeto em si.


## Estrutura



## Exemplo 1

Imaginemos uma classe cuja funcao seja armazenar dados de um plano de exercicios. Cada objeto armazenará uma determinada quantidade de dias, e cada dia deve conter um objetivo, atividades e um número de repeticoes:

In [36]:
from typing import List


class Exercise: 
    
    def __init__(self, name: str, reps: int):
        self.name = name
        self.reps = reps
        
    def __str__(self):
        return f"{self.name} (x{self.reps})"

        
class Day:
    
    def __init__(self, objective: str, exercises: List[Exercise]):
        self.objective = objective
        self.exercises = exercises
    
    def __str__(self):
        exercise_str = '\n'.join(map(str, self.exercises))
        return f"Objective: {self.objective}\nExercises: \n{exercise_str}"

    
class ExercisePlan:
    
    def __init__(self, trainer: str, days: List[Day]):
        self.days = days
        self.trainer = trainer
    
    def __str__(self):
        return self.trainer + '\n\n' + '\n\n'.join(f"Day {i}: \n{str(day)}" for i, day in enumerate(self.days, 1))
    
    
e1 = Exercise('Sit-up', 15)
e2 = Exercise('Leg raise', 15)
e3 = Exercise('Abdominal crunch', 20)
e4 = Exercise('Push-up', 20)
e5 = Exercise('Pull-up', 20)

d1 = Day('Abs', [e1,e2,e3])
d2 = Day('Chest', [e4, e5])

p = ExercisePlan('Paulo Cintura', [d1, d2])

print(p)
        

Paulo Cintura

Day 1: 
Objective: Abs
Exercises: 
Sit-up (x15)
Leg raise (x15)
Abdominal crunch (x20)

Day 2: 
Objective: Chest
Exercises: 
Push-up (x20)
Pull-up (x20)


Como podem ver, o processo de criacao do objeto Exercise Plan é bastante tedioso e complicado. A situacao só pioraria se decidirmos adicionar mais propriedades aos exercícios (número de calorias, instrucoes), aos dias (alimentacao, horarios) ou ao próprio plano (duracao, custo).

Vejamos como o padrao Builder pode dar conta dessa complexidade e tornar a criacao do objeto ExercisePlan mais fluida e intuitiva:

In [26]:
from typing import List


class Exercise: 
    
    def __init__(self, name: str, reps: int):
        self.name = name
        self.reps = reps
        
    def __str__(self):
        return f"{self.name} (x{self.reps})"

        
class Day:
    
    def __init__(self, objective: str, exercises: List[Exercise]=[]):
        self.objective = objective
        self.exercises = exercises[::]
        
    def add_exercise(self, exercise: Exercise):
        self.exercises.append(exercise)
    
    def __str__(self):
        exercise_str = '\n\t'.join(map(str, self.exercises))
        return f"Objective: {self.objective}\nExercises: \n\t{exercise_str}"

    
class ExercisePlan:
    
    def __init__(self, student: str, days: List[Day]=[]):
        self.days = days[::]
        self.student = student
        
    def add_day(self, day: Day):
        self.days.append(day)
    
    def __str__(self):
        return self.student + '\n\n' + '\n\n'.join(f"Day {i}: \n{str(day)}" for i, day in enumerate(self.days, 1))


### Aqui comeca a implementacao do Builder
    
    
class ExercisePlanBuilder:
    
    def __init__(self, student: str):
        self.exercise_plan = ExercisePlan(student)
        
    def add_day(self, objective: str) -> 'ExercisePlanBuilder':
        self.exercise_plan.add_day(Day(objective))
        return self

    def add_exercise(self, name: str, reps: int) -> 'ExercisePlanBuilder':
        self.current_day().add_exercise(Exercise(name, reps))
        return self
        
    def current_day(self) -> Day:
        return self.exercise_plan.days[-1]
    
    def build(self) -> ExercisePlan:
        return self.exercise_plan
        

builder = ExercisePlanBuilder('Paulo Cintura')

plan = (builder.add_day('Abs')
           .add_exercise('Sit-up', 15)
           .add_exercise('Leg raise', 15)
           .add_exercise('Abdominal crunch', 20)
       .add_day('Chest')
           .add_exercise('Push-up', 20)
           .add_exercise('Pull-up', 20)
        .build()
       )

print(plan)

Paulo Cintura

Day 1: 
Objective: Abs
Exercises: 
	Sit-up (x15)
	Leg raise (x15)
	Abdominal crunch (x20)

Day 2: 
Objective: Chest
Exercises: 
	Push-up (x20)
	Pull-up (x20)


Nesta implementacao, o Builder é uma classe concreta. Uma boa recomendacao, porém, é criar um Builder abstrato para que o processo de construcao de objetos possa ser reaproveitado em diferentes classes. Suponhamos que nossa academia queira oferecer servicos de nutricionista além de planos de exercício. Com a classe abstrata `PlanBuilder`, podemos permitir essas duas funcionalidades de maneira bastante flexível.

In [27]:
from typing import List
import abc


class Exercise: 
    
    def __init__(self, name: str, reps: int):
        self.name = name
        self.reps = reps
        
    def __str__(self):
        return f"{self.name} (x{self.reps})"

    
class Meal: 
    
    def __init__(self, dish: str, quantity_in_grams: int):
        self.dish = dish
        self.quantity_in_grams = quantity_in_grams
        
    def __str__(self):
        return f"{self.dish} ({self.quantity_in_grams}g)"
    
    
class Day:
    
    def __init__(self, objective: str, exercises: List[Exercise]=[], meals: List[Meal]=[]):
        self.objective = objective
        self.exercises = exercises[::]
        self.meals = meals[::]
        
    def add_exercise(self, exercise: Exercise):
        self.exercises.append(exercise)
        
    def add_meal(self, meal: Meal):
        self.meals.append(meal)
    
    def __str__(self):
        exercise_str = (len(self.exercises) > 0)* '\nExercises: \n\t' + '\n\t'.join(map(str, self.exercises))
        meal_str = (len(self.meals) > 0)* '\nMeals: \n\t' + '\n\t'.join(map(str, self.meals))
        return f"Objective: {self.objective}{exercise_str}{meal_str}"

    
class Plan:
    
    def __init__(self, student: str, days: List[Day]=[]):
        self.days = days[::]
        self.student = student
        
    def add_day(self, day: Day):
        self.days.append(day)
    
    def __str__(self):
        return self.student + '\n\n' + '\n\n'.join(f"Day {i}: \n{str(day)}" for i, day in enumerate(self.days, 1))



    
# Builder abstrato

class PlanBuilder(abc.ABC):
    
    def add_day(self, objective: str) -> 'PlanBuilder':
        self.plan.add_day(Day(objective))
        return self

    def add_exercise(self, name: str, reps: int) -> 'PlanBuilder': 
        return self
    
    def add_meal(self, dish: str, quantity_in_grams: int) -> 'PlanBuilder': 
        return self
    
    @abc.abstractmethod
    def build(self) -> Plan:
        pass

# Builder de exercícios (sem mudancas)

class ExercisePlanBuilder(PlanBuilder):

    def __init__(self, student: str):
        self.plan = Plan(student)
        
    def add_exercise(self, name: str, reps: int) -> 'PlanBuilder':
        self.current_day().add_exercise(Exercise(name, reps))
        return self
        
    def current_day(self) -> Day:
        return self.plan.days[-1]
    
    def build(self) -> Plan:
        return self.plan
        

exercise_plan_builder = ExercisePlanBuilder('Paulo Cintura')

exercise_plan = (exercise_plan_builder.add_day('Abs')
           .add_exercise('Sit-up', 15)
           .add_exercise('Leg raise', 15)
           .add_exercise('Abdominal crunch', 20)
       .add_day('Chest')
           .add_exercise('Push-up', 20)
           .add_exercise('Pull-up', 20)
        .build()
       )

print(exercise_plan)

Paulo Cintura

Day 1: 
Objective: Abs
Exercises: 
	Sit-up (x15)
	Leg raise (x15)
	Abdominal crunch (x20)

Day 2: 
Objective: Chest
Exercises: 
	Push-up (x20)
	Pull-up (x20)


Nosso `ExercisePlanBuilder` continua igual, mas podemos usar a flexibilidade do `PlanBuilder` para oferecer planos de alimentacao...

In [25]:
class MealPlanBuilder(PlanBuilder):
    
    def __init__(self, student: str):
        self.plan = Plan(student)
    
    def add_meal(self, dish: str, quantity_in_grams: int) -> 'PlanBuilder':
        self.current_day().add_meal(Meal(dish, quantity_in_grams))
        return self
        
    def current_day(self) -> Day:
        return self.plan.days[-1]
    
    def build(self) -> Plan:
        return self.plan
        

meal_plan_builder = MealPlanBuilder('Homer')

meal_plan = (meal_plan_builder.add_day('Abs')
           .add_meal('Donuts', 500)
           .add_meal('Beer', 1000)
       .add_day('Abs')
           .add_meal('Burger', 600)
           .add_meal('Milkshake', 500)
        .build()
       )

print(meal_plan)

Homer

Day 1: 
Objective: Abs
Meals: 
	Donuts (500g)
	Beer (1000g)

Day 2: 
Objective: Abs
Meals: 
	Burger (600g)
	Milkshake (500g)


E também planos combinados de alimentacao + exercício:

In [28]:
class BodyBuildingPlanBuilder(PlanBuilder):
    
    def __init__(self, student: str):
        self.plan = Plan(student)
    
    def add_meal(self, dish: str, quantity_in_grams: int) -> 'PlanBuilder':
        self.current_day().add_meal(Meal(dish, quantity_in_grams))
        return self
    
    def add_exercise(self, name: str, reps: int) -> 'PlanBuilder':
        self.current_day().add_exercise(Exercise(name, reps))
        return self
        
    def current_day(self) -> Day:
        return self.plan.days[-1]
    
    def build(self) -> Plan:
        return self.plan
        

body_building_plan_builder = BodyBuildingPlanBuilder('Gracyanne')

body_building_plan = (body_building_plan_builder.add_day('Legs')
           .add_meal('Sweet potato', 1000)
           .add_exercise('Leg press', 200)
       .add_day('Butt')
           .add_meal('Chicken breast', 1000)
           .add_exercise('Bulgarian squats', 200)
        .build()
       )

print(body_building_plan)

Gracyanne

Day 1: 
Objective: Legs
Exercises: 
	Leg press (x200)
Meals: 
	Sweet potato (1000g)

Day 2: 
Objective: Butt
Exercises: 
	Bulgarian squats (x200)
Meals: 
	Chicken breast (1000g)


## Exemplo 2: Builder com Composite

Para nosso segundo exemplo, retomaremos a classe `Cluster` originalmente usada para demonstrar o padrao Composite.

In [1]:
import abc
from itertools import cycle
from string import ascii_lowercase
from collections import Counter


class Server(abc.ABC):
    """
    Application the counts the number of letter in a piece of text.
    """
    
    valid_chars = set(ascii_lowercase)
    
    def __repr__(self):
        return self.id_
    
    def __hash__(self):
        return hash(self.id_)
    
    @property
    def cpu(self) -> int:
        """
        Number of CPUs.
        """
        return self._cpu
    
    @property
    def mem(self) -> int:
        """
        Total memory.
        """
        return self._mem
    
    @property
    def id_(self) -> str:
        """
        Server id.
        """
        return self._id
    
    def display_svr_info(self):
        print(f"ID: {self.id_}, MEM: {self.mem}, CPU: {self.cpu}")
    
    def serve(self, req: dict) -> dict:
        """
        Count the number of latter in req.
        """
        text = req["text"].lower()
        text = [char for char in text if char in self.valid_chars]
        ctr = Counter(text)
        print(self.id_, ctr)
        return ctr
    
class Worker(Server):
    """
    A single working node.
    """
    def __init__(self, id_: str, mem: int, cpu: int):
        self._mem = mem
        self._cpu = cpu
        self._id = id_
        
class Cluster(Server):
    """
    A cluster of nodes or clusters.
    """
    def __init__(self, id_: str, *servers: 'Server'):
        self._id = id_
        self._servers = set(servers)
        self._splitter = None
        
    @property
    def cpu(self) -> int:
        return sum(svr.cpu for svr in self._servers)
    
    @property
    def mem(self) -> int:
        return sum(svr.mem for svr in self._servers)
    
    def _create_splitter(self):
        """
        Creates an object that cycles the servers.
        """
        def iterator():
            for worker in cycle(self._servers):
                req = yield
                worker.serve(req)
        splitter = iterator()
        splitter.send(None)
        return splitter
    
    def serve(self, req: dict) -> dict:
        if not self._splitter:
            self._splitter = self._create_splitter()
            
        self._splitter.send(req)
        
    def add(self, *servers: 'Server'):
        self._servers |= set(servers)
        self._splitter = None
        
    def remove(self, *servers: 'Server'):
        self._servers -= set(servers)
        self._splitter = None

No exemplo original, o processo para a criacao de um cluster de oito workers era este: 

In [5]:
workers = [Worker(f"worker{i}", i*2, i)for i in range(8)]

cluster = Cluster("cluster")
cluster.add(*workers)
cluster.display_svr_info()


ID: cluster, MEM: 56, CPU: 28


Com o Builder, o processo de criacao ficaria assim: 

In [7]:
class Builder(abc.ABC):
    
    def __init__(self, name: str):
        self.cluster = Cluster(name)
    
    def add_worker(self, _id: str, mem: int, cpu: int ) -> 'Builder':
        return self
    
    @abc.abstractmethod
    def build(self) -> Cluster:
        pass

class ClusterBuilder(Builder):
    
    def add_worker(self, _id: str, mem: int, cpu: int) -> Builder:
        self.cluster.add(Worker(_id, mem, cpu))
        return self
    
    def build(self) -> Cluster:
        return self.cluster
    

cb = ClusterBuilder('cluster')
for i in range(8):
    cb.add_worker(f"worker{i}", i*2, i)
cluster = cb.build()

cluster.display_svr_info()

ID: cluster, MEM: 56, CPU: 28


Com a mudanca, o cluster só é exposto ao cliente quando já contém todos os servidores e o processo de criacao está completo.

## Prós e contras:

### Pros
- Permite maior robustez na criacao de objetos complexos
- Cria uma separacao clara entre a construcao do objeto e o objeto em si
- Evita construtores ou factory methods com muitos parâmetros

### Cons
- Aumenta a complexidade do sistema como um todo
- Duplicacao de código (propriedades, getters, etc) entre Builder e Product
- Exige que o cliente conheca a implementacao do objeto

## Discussao:

Like the Abstract Factory pattern, the Builder pattern requires that you define an interface, which will be used by clients to create complex objects in pieces. In the MazeBuilder example, there are BuildMaze(), BuildRoom() and BuildDoor() methods, along with a GetMaze() method. How does the Builder pattern allow one to add new methods to the Builder's interface, without having to change each and every sub-class of the Builder?